# <div align="center">Machine Learning: 21 Blackjack</div>

<div style="text-align: center;">
  <img src="./recursos_modelo/portada.jpg" width="800" height="450" style="object-fit: cover;" />
</div>

*Jorge Alonso Conde - Trabajo final Machine Learning*

# Importación del data set #

In [37]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#ML
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from xgboost import XGBClassifier

#Windgets y botones
import ipywidgets as widgets
from IPython.display import display, clear_output

#Exportar el gid y modelo en archivo .pkl para poder importarlo en streamlit
import joblib


In [2]:
ds_blackjack = pd.read_csv("../data/blkjckhands.csv", nrows=200004) #menos registros

In [3]:
ds_blackjack.tail(20)

,Unnamed: 0,PlayerNo,card1,card2,card3,card4,card5,sumofcards,dealcard1,dealcard2,...,dealcard4,dealcard5,sumofdeal,blkjck,winloss,plybustbeat,dlbustbeat,plwinamt,dlwinamt,ply2cardsum
199984,4,Player5,3,8,9,0,0,20,4,5,...,0,0,17,nowin,Win,Plwin,Beat,20,0,11
199985,5,Player6,10,11,0,0,0,21,4,5,...,0,0,17,Win,Win,Plwin,Beat,25,0,21
199986,0,Player1,4,2,2,10,0,18,10,10,...,0,0,20,nowin,Loss,Beat,Dlwin,0,10,6
199987,1,Player2,7,3,10,0,0,20,10,10,...,0,0,20,nowin,Push,Push,Push,10,0,10
199988,2,Player3,10,3,0,6,0,19,10,10,...,0,0,20,nowin,Loss,Beat,Dlwin,0,10,13
199989,3,Player4,9,4,0,4,0,17,10,10,...,0,0,20,nowin,Loss,Beat,Dlwin,0,10,13
199990,4,Player5,11,1,5,0,0,17,10,10,...,0,0,20,nowin,Loss,Beat,Dlwin,0,10,12
199991,5,Player6,8,10,0,0,0,18,10,10,...,0,0,20,nowin,Loss,Beat,Dlwin,0,10,18
199992,0,Player1,7,5,0,1,10,23,10,10,...,0,0,20,nowin,Loss,Bust,PlBust,0,10,12
199993,1,Player2,11,10,0,0,0,21,10,10,...,0,0,20,Win,Win,Plwin,Beat,25,0,21


# Estudio del Data Set #

#### Este estudio se centra en cómo juegan los jugadores y en mejorar la toma de decisiones durante la partida. Por lo tanto, se eliminarán las columnas que hacen referencia a las apuestas, ya que no son necesarias para este análisis. ####

In [4]:
ds_blackjack.head()

,Unnamed: 0,PlayerNo,card1,card2,card3,card4,card5,sumofcards,dealcard1,dealcard2,...,dealcard4,dealcard5,sumofdeal,blkjck,winloss,plybustbeat,dlbustbeat,plwinamt,dlwinamt,ply2cardsum
0,0,Player1,7,10,0,0,0,17,10,8,...,0,0,18,nowin,Loss,Beat,Dlwin,0,10,17
1,1,Player2,10,9,0,0,0,19,10,8,...,0,0,18,nowin,Win,Plwin,Beat,20,0,19
2,2,Player3,9,8,0,0,0,17,10,8,...,0,0,18,nowin,Loss,Beat,Dlwin,0,10,17
3,3,Player4,2,10,0,5,0,17,10,8,...,0,0,18,nowin,Loss,Beat,Dlwin,0,10,12
4,4,Player5,10,2,0,5,0,17,10,8,...,0,0,18,nowin,Loss,Beat,Dlwin,0,10,12


In [5]:
ds_blackjack.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200004 entries, 0 to 200003
Data columns (total 21 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Unnamed: 0   200004 non-null  int64 
 1   PlayerNo     200004 non-null  object
 2   card1        200004 non-null  int64 
 3   card2        200004 non-null  int64 
 4   card3        200004 non-null  int64 
 5   card4        200004 non-null  int64 
 6   card5        200004 non-null  int64 
 7   sumofcards   200004 non-null  int64 
 8   dealcard1    200004 non-null  int64 
 9   dealcard2    200004 non-null  int64 
 10  dealcard3    200004 non-null  int64 
 11  dealcard4    200004 non-null  int64 
 12  dealcard5    200004 non-null  int64 
 13  sumofdeal    200004 non-null  int64 
 14  blkjck       200004 non-null  object
 15  winloss      200004 non-null  object
 16  plybustbeat  200004 non-null  object
 17  dlbustbeat   200004 non-null  object
 18  plwinamt     200004 non-null  int64 
 19  dl

In [6]:
ds_blackjack.describe()

,Unnamed: 0,card1,card2,card3,card4,card5,sumofcards,dealcard1,dealcard2,dealcard3,dealcard4,dealcard5,sumofdeal,plwinamt,dlwinamt,ply2cardsum
count,200004.000000,200004.000000,200004.000000,200004.000000,200004.000000,200004.000000,200004.000000,200004.000000,200004.000000,200004.000000,200004.000000,200004.000000,200004.000000,200004.000000,200004.000000,200004.000000
mean,2.500000,7.129147,7.086298,1.956026,2.352258,0.393777,18.917507,7.209906,7.133767,4.364493,1.373043,0.225535,20.306744,9.755005,4.760305,14.215446
std,1.707829,2.996806,3.008780,3.533596,3.661872,1.739657,3.209495,2.974229,2.986018,4.083142,3.033778,1.317447,2.555308,9.790661,4.994264,4.429589
min,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,9.000000,1.000000,1.000000,0.000000,0.000000,0.000000,10.000000,0.000000,0.000000,2.000000
25%,1.000000,5.000000,4.000000,0.000000,0.000000,0.000000,17.000000,5.000000,5.000000,0.000000,0.000000,0.000000,18.000000,0.000000,0.000000,12.000000
50%,2.500000,8.000000,8.000000,0.000000,0.000000,0.000000,19.000000,8.000000,8.000000,4.000000,0.000000,0.000000,20.000000,10.000000,0.000000,14.000000
75%,4.000000,10.000000,10.000000,2.000000,4.000000,0.000000,21.000000,10.000000,10.000000,9.000000,0.000000,0.000000,22.000000,20.000000,10.000000,18.000000
max,5.000000,11.000000,11.000000,11.000000,10.000000,10.000000,26.000000,11.000000,11.000000,11.000000,10.000000,10.000000,26.000000,25.000000,10.000000,21.000000


### Data set: variables y descripción (ACTUALIZAR)

| Variable         | Traducción                        | Significado                                                                 |
|------------------|------------------------------------|------------------------------------------------------------------------------|
| `Unnamed: 0`     | Índice                             | Número de fila del dataset (índice automático del archivo CSV)              |
| `PlayerNo`       | Jugador N.º                        | Identificador del jugador (cadena de texto)                                 |
| `card1`          | Carta 1                            | Primera carta recibida por el jugador                                       |
| `card2`          | Carta 2                            | Segunda carta recibida por el jugador                                       |
| `card3`          | Carta 3                            | Tercera carta del jugador (si pide carta)                                   |
| `card4`          | Carta 4                            | Cuarta carta del jugador (si sigue pidiendo)                                |
| `card5`          | Carta 5                            | Quinta carta del jugador (última posible carta recibida)                    |
| `sumofcards`     | Suma de cartas jugador             | Suma total de las cartas del jugador                                        |
| `dealcard1`      | Carta crupier 1                    | Primera carta visible del crupier                                           |
| `dealcard2`      | Carta crupier 2                    | Segunda carta del crupier                                                   |
| `dealcard3`      | Carta crupier 3                    | Tercera carta del crupier (si pide carta)                                   |
| `dealcard4`      | Carta crupier 4                    | Cuarta carta del crupier (si sigue pidiendo)                                |
| `dealcard5`      | Carta crupier 5                    | Quinta carta del crupier (última posible)                                   |
| `sumofdeal`      | Suma de cartas crupier             | Suma total de las cartas del crupier                                        |
| `blkjck`         | Blackjack                          | Indica si hubo blackjack natural (21 con las dos primeras cartas)           |
| `winloss`        | Resultado                          | Resultado de la mano para el jugador (`Win`, `Loss`, `Push`)          |
| `plybustbeat`    | Jugador se pasó                    | Resultado de la partida para el jugador: Beat, DlBust, Plwin, Bust, Push                      |
| `dlbustbeat`     | Crupier se pasó                    | Resultado de la partida para el dealer: Dlwin, Bust, Beat, PlBust, Push                   |
| `ply2cardsum`    | Suma 2 primeras cartas jugador     | Suma de las primeras dos cartas del jugador                                 |



In [7]:
ds_blackjack["winloss"].value_counts()

winloss
Loss    95208
Win     86094
Push    18702
Name: count, dtype: int64

In [8]:
ds_blackjack["plybustbeat"].value_counts()

#Beat	= El jugador gana con una mano superior sin que nadie se pase de 21.
#DlBust	= El jugador gana porque el crupier se pasó de 21.
#Plwin	= El jugador gana con un blackjack natural (As + 10).
#Bust	= El jugador pierde porque se pasó de 21.
#Push	= Empate: el jugador y el crupier tienen el mismo total.

plybustbeat
Beat      59270
DlBust    48628
Plwin     37466
Bust      35938
Push      18702
Name: count, dtype: int64

In [9]:
ds_blackjack["dlbustbeat"].value_counts()

#Dlwin	El crupier gana con una mano superior sin que nadie se pase de 21.
#Bust	El crupier pierde porque se pasó de 21.
#Beat	El crupier gana porque el jugador se pasó de 21.
#PlBust	El jugador pierde por pasarse, lo que implica victoria para el crupier.
#Push	Empate: el crupier y el jugador tienen el mismo total.

dlbustbeat
Dlwin     59270
Bust      48628
Beat      37466
PlBust    35938
Push      18702
Name: count, dtype: int64

In [10]:
ds_blackjack["winloss"].value_counts()

winloss
Loss    95208
Win     86094
Push    18702
Name: count, dtype: int64

In [11]:
no_cuenta_bj = ds_blackjack[ds_blackjack["ply2cardsum"] != 21] 

conteo_sumofdeal_filtrado = no_cuenta_bj["sumofdeal"].value_counts().sort_index()

conteo_sumofdeal_filtrado


sumofdeal
10        6
11       12
12       53
13       67
14      124
15      194
16      330
17    27071
18    26441
19    25376
20    34032
21    23047
22    14068
23    12148
24    10725
25     9070
26     7691
Name: count, dtype: int64

# Limpieza y trasformación del Data Frame #

In [12]:
#Eliminamos columnas referente a apuestas

ds_blackjack = ds_blackjack.drop(["plwinamt", "dlwinamt"], axis=1)



In [13]:
# Suma dos primeras cartas del jugador
ds_blackjack = ds_blackjack.rename(columns= {"player_2cards_sum": "ply2cardsum"})

In [14]:
# número de cartas totales pedidas por el jugador (cuenta en una única fila, el numero de cartas que ha pedido el jugador)

ds_blackjack["ply_No_cards"] = ds_blackjack[["card1", "card2", "card3", "card4", "card5"]].ne(0).sum(axis=1) #.e(0) quiere decir not equal 0

# número de cartas totales pedidas por el dealer (cuenta en una única fila, el numero de cartas que ha pedido el dealer)

ds_blackjack["deal_No_cards"] = ds_blackjack[["dealcard1", "dealcard2", "dealcard3", "dealcard4", "dealcard5"]].ne(0).sum(axis=1)

In [15]:
# Suma total de cartas visibles en la mesa al empezar la partida (player vs crepier)

ds_blackjack["deal_2cards_sum"] = ds_blackjack[["dealcard1", "dealcard2"]].sum(axis=1)

In [16]:
# Suma total de cartas visibles en la mesa al empezar la partida (player vs crepier)

ds_blackjack["sum_3first_cards"] = ds_blackjack[["card1", "card2", "dealcard1"]].sum(axis=1)



Pasamos la variable winloss a numérica:

In [17]:
#trasformar las variables categoricas en numericas, la mas importante win, los, push

def  numeric_def (x):
    if x == "Loss":
        return 0
    if x == "Push":
        return 1
    if x == "Win":
        return 2
    

ds_blackjack["winloss_numeric"] = ds_blackjack["winloss"].apply(numeric_def)


Tabla de variable:

| Valor original | Significado          | Valor asignado |
| -------------- | -------------------- | -------------- |
| **Loss**       | Derrota del jugador  | 0              |
| **Push**       | Empate               | 1              |
| **Win**        | Victoria del jugador | 2              |

Pasamos la variable blkjck a numérica:

In [18]:
#Cambiamos la columna blkjck a numerica (esta categorica indica si el jugador tiene un 21 únicamente con dos cartas)

print(ds_blackjack["blkjck"].describe())
print("----")
print(ds_blackjack["blkjck"].value_counts())

count     200004
unique         2
top        nowin
freq      190455
Name: blkjck, dtype: object
----
blkjck
nowin    190455
Win        9549
Name: count, dtype: int64


In [19]:
# Creamos la función y la aplicamos en un nueva columna

def blkjck_def (x):
    if x == "nowin":
        return 0
    if x == "Win":
        return 1
    
ds_blackjack["blkjck_numeric"] = ds_blackjack["blkjck"].apply(blkjck_def)

Tabla de variable:

| Valor original | Significado                        | Valor asignado |
| -------------- | ---------------------------------- | -------------- |
| **nowin**      | No se ganó por blackjack           | 0              |
| **Win**        | Victoria del jugador por blackjack | 1              |


Pasamos la variable plybustbeat a numerica:

In [20]:
#Entendemos como se compora la columna plybustbeat

print(ds_blackjack["plybustbeat"].describe())
print("----")
print(ds_blackjack["plybustbeat"].value_counts())

count     200004
unique         5
top         Beat
freq       59270
Name: plybustbeat, dtype: object
----
plybustbeat
Beat      59270
DlBust    48628
Plwin     37466
Bust      35938
Push      18702
Name: count, dtype: int64


In [21]:
# Creamos la columna en formato numerica

def plybustbeat_def (x):
    if x == "Push":
        return 0
    if x == "Plwin":
        return 1
    if x == "DlBust":
        return 2
    if x == "Beat":
        return 3
    if x == "Bust":
        return 4
    
ds_blackjack["plybustbeat_numeric"] = ds_blackjack["plybustbeat"].apply(plybustbeat_def)

Tabla de variable:

| Valor original | Significado                      | Valor asignado |
| -------------- | -------------------------------- | -------------- |
| **Push**       | Empate                           | 0              |
| **Plwin**      | Gana el jugador (mejor mano)     | 1              |
| **DlBust**     | Gana el jugador (dealer se pasa) | 2              |
| **Beat**       | Gana el dealer (mejor mano)      | 3              |
| **Bust**       | Pierde el jugador (se pasa)      | 4              |

In [22]:
ds_blackjack

,Unnamed: 0,PlayerNo,card1,card2,card3,card4,card5,sumofcards,dealcard1,dealcard2,...,plybustbeat,dlbustbeat,ply2cardsum,ply_No_cards,deal_No_cards,deal_2cards_sum,sum_3first_cards,winloss_numeric,blkjck_numeric,plybustbeat_numeric
0,0,Player1,7,10,0,0,0,17,10,8,...,Beat,Dlwin,17,2,2,18,27,0,0,3
1,1,Player2,10,9,0,0,0,19,10,8,...,Plwin,Beat,19,2,2,18,29,2,0,1
2,2,Player3,9,8,0,0,0,17,10,8,...,Beat,Dlwin,17,2,2,18,27,0,0,3
3,3,Player4,2,10,0,5,0,17,10,8,...,Beat,Dlwin,12,3,2,18,22,0,0,3
4,4,Player5,10,2,0,5,0,17,10,8,...,Beat,Dlwin,12,3,2,18,22,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199999,1,Player2,9,3,0,7,0,19,7,10,...,Plwin,Beat,12,3,2,17,19,2,0,1
200000,2,Player3,1,5,10,3,0,19,7,10,...,Plwin,Beat,6,4,2,17,13,2,0,1
200001,3,Player4,2,7,6,10,0,25,7,10,...,Bust,PlBust,9,4,2,17,16,0,0,4
200002,4,Player5,11,8,0,0,0,19,7,10,...,Plwin,Beat,19,2,2,17,26,2,0,1


In [23]:
ds_blackjack.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200004 entries, 0 to 200003
Data columns (total 26 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   Unnamed: 0           200004 non-null  int64 
 1   PlayerNo             200004 non-null  object
 2   card1                200004 non-null  int64 
 3   card2                200004 non-null  int64 
 4   card3                200004 non-null  int64 
 5   card4                200004 non-null  int64 
 6   card5                200004 non-null  int64 
 7   sumofcards           200004 non-null  int64 
 8   dealcard1            200004 non-null  int64 
 9   dealcard2            200004 non-null  int64 
 10  dealcard3            200004 non-null  int64 
 11  dealcard4            200004 non-null  int64 
 12  dealcard5            200004 non-null  int64 
 13  sumofdeal            200004 non-null  int64 
 14  blkjck               200004 non-null  object
 15  winloss              200004 non-nu

In [24]:
ds_blackjack.describe()

,Unnamed: 0,card1,card2,card3,card4,card5,sumofcards,dealcard1,dealcard2,dealcard3,...,dealcard5,sumofdeal,ply2cardsum,ply_No_cards,deal_No_cards,deal_2cards_sum,sum_3first_cards,winloss_numeric,blkjck_numeric,plybustbeat_numeric
count,200004.000000,200004.000000,200004.000000,200004.000000,200004.000000,200004.000000,200004.000000,200004.000000,200004.000000,200004.000000,...,200004.000000,200004.000000,200004.000000,200004.000000,200004.000000,200004.000000,200004.000000,200004.000000,200004.000000,200004.000000
mean,2.500000,7.129147,7.086298,1.956026,2.352258,0.393777,18.917507,7.209906,7.133767,4.364493,...,0.225535,20.306744,14.215446,2.697991,2.891342,14.343673,21.425351,0.954431,0.047744,2.281374
std,1.707829,2.996806,3.008780,3.533596,3.661872,1.739657,3.209495,2.974229,2.986018,4.083142,...,1.317447,2.555308,4.429589,0.764366,0.809928,4.388708,5.331372,0.951010,0.213225,1.223636
min,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,9.000000,1.000000,1.000000,0.000000,...,0.000000,10.000000,2.000000,2.000000,2.000000,2.000000,3.000000,0.000000,0.000000,0.000000
25%,1.000000,5.000000,4.000000,0.000000,0.000000,0.000000,17.000000,5.000000,5.000000,0.000000,...,0.000000,18.000000,12.000000,2.000000,2.000000,12.000000,18.000000,0.000000,0.000000,1.000000
50%,2.500000,8.000000,8.000000,0.000000,0.000000,0.000000,19.000000,8.000000,8.000000,4.000000,...,0.000000,20.000000,14.000000,3.000000,3.000000,14.000000,22.000000,1.000000,0.000000,2.000000
75%,4.000000,10.000000,10.000000,2.000000,4.000000,0.000000,21.000000,10.000000,10.000000,9.000000,...,0.000000,22.000000,18.000000,3.000000,3.000000,18.000000,25.000000,2.000000,0.000000,3.000000
max,5.000000,11.000000,11.000000,11.000000,10.000000,10.000000,26.000000,11.000000,11.000000,11.000000,...,10.000000,26.000000,21.000000,5.000000,5.000000,21.000000,32.000000,2.000000,1.000000,4.000000


#### Careación de nuevas columnas calculadas y renombrar columnas del data set ####

# Machine Learning, matriz de confisión y score #

#### Matriz de confusión ####

En este modelo de predicción co aplicación al blackjack, lo que buscamos es predecir la probabilidad de ganar o perder una partida en función de las cartas vistas en la mesa, es decir, en función de las cartas del jugador y la primera carta del crupier (carta visible para el jugador).

Teniendo en cuenta estos factores y la probabilidad de ganar o peder con la mano del jugador, el modelo recomendará pedir carta o plantarse. 

Para ello hay que crear una matriz de confusión asumiento las dos predicciones que va a realizar el modelo: ganar o perder.

|                  | **Predicción: Win**   | **Prediccción: No Win** |
| ---------------- | ------------------- | -------------------- |
| **Real: Win**    | True Positive (TP)  | False Negative (FN)  |
| **Real: No Win** | False Positive (FP) | True Negative (TN)   |


Teniendo en cuenta esta matriz de confisión. Lo que nos interesa en este modelo de machine learning es optimizar el verdadero positivo (TP), y para ello deberemos tenber en cuenta la métrica de Precisión*.<br>
También utizaremos el modelo F1-Score (mezcla entre Precision y Recall*). Esta métrica lo que nos ayuda es a balancear riesgo vs. beneficio en la toma de decisiones.<br>
<br>
*La métrica de Recall, busca detectar con precisión todos los casos positivos.<br>
*Recall = TP/(TP+FN) Elementos que relamente eran positivos y fueron bien identificados<br>
*Precisión = TP/(TP+FP) Elementos que se han predecido positivos y realmente lo eran

### 1 - Dividimos el data set para prepararlos en nustros diferentes modelos de predicción ###

In [25]:
#Para realzar el modelo de predicción solo debemos poner variables que el jugador pueda completar con la información que tiene en el momento del juego

X = ds_blackjack[["sumofcards", "dealcard1", "ply2cardsum", "ply_No_cards"]]

y = ds_blackjack["winloss_numeric"]

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y) #Estratify=y nos ayuda a conservar las proporciones de las clases al hacer el train test split, esto es mejor para que las clases con menos datos no desaparezcan

### 2 - Creamos los diferentes modelos de predicción ###

Random Forest (V1)

In [27]:
#Creamso el pipeline para el random fores

pipeline_randomforest = Pipeline([
    ('clf', RandomForestClassifier(
        n_estimators=100,
        max_depth=None,
        random_state=42,
        class_weight='balanced'))  #Balancea las clases, se utiliza en rf y xgboost porque no están estandarizadas
        ])

In [28]:
#Creamos un param grid para realizar pruebas con diferentes parametros y creamos la validación cruzada. Scoring de F1 (primera prueba)

param_grid = {
    'clf__n_estimators': [100, 200],
    'clf__max_depth': [None, 10, 20],
    'clf__min_samples_split': [2, 5],
    'clf__min_samples_leaf': [1, 2]
}

grid = GridSearchCV(estimator= pipeline_randomforest, param_grid=param_grid, scoring="f1_macro", cv=5, verbose=1, n_jobs= -1)

In [29]:
grid.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


,estimator,Pipeline(step...m_state=42))])
,param_grid,"{'clf__max_depth': [None, 10, ...], 'clf__min_samples_leaf': [1, 2], 'clf__min_samples_split': [2, 5], 'clf__n_estimators': [100, 200]}"
,scoring,'f1_macro'
,n_jobs,-1
,refit,True
,cv,5
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,n_estimators,200


In [30]:
#Comprobamos resultados del modelo y los mejores parametros

print("Mejor modelo basado en F1 macro:")
print(grid.best_params_)
print("Mejor F1 promedio en validación cruzada:", grid.best_score_)


Mejor modelo basado en F1 macro:
{'clf__max_depth': 10, 'clf__min_samples_leaf': 2, 'clf__min_samples_split': 2, 'clf__n_estimators': 200}
Mejor F1 promedio en validación cruzada: 0.5554382282988285


In [31]:
#Probamos con el y test

y_pred_rf = grid.predict(X_test)
print("\nEvaluación en test set:")
print(classification_report(y_test, y_pred_rf))


Evaluación en test set:
              precision    recall  f1-score   support

           0       0.78      0.72      0.75     19042
           1       0.20      0.61      0.30      3740
           2       0.80      0.51      0.62     17219

    accuracy                           0.62     40001
   macro avg       0.59      0.61      0.56     40001
weighted avg       0.74      0.62      0.65     40001



Podemos observar que el modelo predice correctamente en la mayoría de los casos correctamente Las victorias y derrotas, el problema está en los empates. Al ser un juego de azar, y el turno del dealer es posterior al tuyo, el muy dificil predecir empates.

A pesar de ello vemos que hay una predicción del 62% correcto segun el F1 Score. Este baja un poco debido al recall, ya que ha muchas victorias reales que no se predijeron.
Por otro lado, las derrotas se han predicho muy bien ya que el F1-Score nos marca un 75% de derrotas predichas. 

In [32]:
y_pred_rf #Predicciones realizadas por nuestro modelo.

array([2, 0, 1, ..., 2, 0, 0], shape=(40001,))

In [33]:
def winloss_predict (x):
    if x == 0:
        return "Loss"
    if x == 1:
        return "Push"
    if x == 2:
        return "Win"
    


In [34]:
resultado_predicciones_rf = [winloss_predict(x) for x in y_pred_rf]

In [35]:
resultado_predicciones_rf

['Win',
 'Loss',
 'Push',
 'Win',
 'Loss',
 'Push',
 'Win',
 'Loss',
 'Push',
 'Win',
 'Loss',
 'Win',
 'Push',
 'Win',
 'Win',
 'Loss',
 'Loss',
 'Loss',
 'Push',
 'Push',
 'Win',
 'Push',
 'Push',
 'Loss',
 'Push',
 'Win',
 'Loss',
 'Loss',
 'Win',
 'Win',
 'Win',
 'Loss',
 'Loss',
 'Win',
 'Loss',
 'Push',
 'Loss',
 'Win',
 'Win',
 'Push',
 'Loss',
 'Win',
 'Loss',
 'Push',
 'Push',
 'Push',
 'Push',
 'Push',
 'Loss',
 'Loss',
 'Push',
 'Loss',
 'Win',
 'Win',
 'Push',
 'Loss',
 'Loss',
 'Push',
 'Loss',
 'Loss',
 'Push',
 'Push',
 'Loss',
 'Loss',
 'Push',
 'Win',
 'Loss',
 'Loss',
 'Loss',
 'Loss',
 'Push',
 'Loss',
 'Push',
 'Loss',
 'Win',
 'Loss',
 'Loss',
 'Push',
 'Push',
 'Push',
 'Loss',
 'Loss',
 'Push',
 'Win',
 'Loss',
 'Push',
 'Loss',
 'Push',
 'Push',
 'Push',
 'Loss',
 'Loss',
 'Loss',
 'Win',
 'Loss',
 'Win',
 'Push',
 'Loss',
 'Push',
 'Win',
 'Push',
 'Push',
 'Loss',
 'Loss',
 'Win',
 'Push',
 'Loss',
 'Loss',
 'Win',
 'Loss',
 'Push',
 'Win',
 'Win',
 'Push',
 '

### 3 - Elección del modelo ganador ###

#### Tabla comparación F1 en los diferentes modelos: 

| Modelo           | F1 Loss (0) | F1 Push (1) | F1 Win (2) | F1 Macro |
| ---------------- | ----------- | ----------- | ---------- | -------- |
| Random Forest V1 | 0.75        | 0.30        | 0.62       | 0.56     |
| Random Forest V2 | 0.74        | 0.30        | 0.63       | 0.56     |
| XGBoost          | 0.78        | 0.08        | 0.69       | 0.52     |

---

A la hora de elegir el mejor modelo entre los tres planteados, nos guiaremos por las métricas obtenidas en el conjunto de test de cada uno de ellos.

Lo primero que destaca es que el modelo **XGBoost** casi no detecta los empates (*pushes*), ya que presenta una métrica F1 de **0.08** en esta clase.  
En el blackjack, los empates tienen mucho menor peso que las victorias o derrotas y, al jugador, no le afectan directamente, ya que recupera lo apostado en caso de que se estén realizando apuestas.  
A pesar de ello, es importante anticiparse y poder predecir estos empates. Si el modelo los clasifica erróneamente como *losses* y decides pedir una carta, asumes un mayor riesgo de pasarte de 21 y perder la partida.

Por este motivo, queda descartado el modelo de predicción **XGBoost**.

---

#### Comparativa entre Random Forest V1 y V2

Al comparar los dos modelos **Random Forest**, vemos que las métricas F1 son prácticamente idénticas, a pesar de que el segundo modelo utiliza más variables.  
La métrica que nos hace preferir la versión 1 es el **Recall** de la clase *Push* (empate):  
- **Random Forest V1:** 0.61  
- **Random Forest V2:** 0.54  

Esto indica que V1 detectará con mayor sensibilidad los empates reales. El resto de métricas son prácticamente iguales.

---

#### Modelo seleccionado

El modelo seleccionado para crear la herramienta de predicción de blackjack es:  
**Random Forest (V1)**.


### 3 - Botón de predicciones instantáneas ###

In [36]:
# Se nombra el modelo elegido con el que se va a realizar la presicción
grid_cv = grid
model = grid_cv.best_estimator_

# Diccionario con las 3 posibilidades de resultado en la partida
label_map = {0: "Loss", 1: "Push", 2: "Win"}

def get_probabilities_safe(model, X):
    """
    Devuelve (proba, classes, aproximadas)
    - proba: ndarray shape (n_samples, n_classes)
    - classes: ndarray de clases en el mismo orden que proba
    - aproximadas: bool, True si se usó decision_function + softmax
    """
    # 1) Probabilidades reales si existen
    if hasattr(model, "predict_proba"):
        proba = model.predict_proba(X)
        classes = getattr(model, "classes_", None)
        return proba, classes, False

    # 2) Fallback con decision_function -> softmax (no calibradas)
    if hasattr(model, "decision_function"):
        scores = model.decision_function(X)

        # Binario: puede venir shape (n_samples,); lo convertimos a 2 columnas
        if scores.ndim == 1:
            scores = np.column_stack([-scores, scores])

        # Softmax estable
        scores = scores - scores.max(axis=1, keepdims=True)
        exp_s = np.exp(scores)
        proba = exp_s / exp_s.sum(axis=1, keepdims=True)

        # Clases (si existen). Si no, inferimos 0..n-1
        classes = getattr(model, "classes_", np.arange(proba.shape[1]))
        return proba, classes, True

    # 3) Último recurso: repartir uniformemente
    classes = getattr(model, "classes_", np.array([0,1,2]))
    n_classes = len(classes)
    proba = np.full((len(X), n_classes), 1.0 / n_classes)
    return proba, classes, True


# Widgets para que el usuario cree sus propias predicciones
w_sumofcards   = widgets.IntText(placeholder='Introduzca un número', description='sumofcards:')
w_dealcard1    = widgets.IntText(placeholder='Introduzca un número', description='dealcard1:')
w_ply2cardsum  = widgets.IntText(placeholder='Introduzca un número', description='ply2cardsum:')
w_ply_No_cards = widgets.IntText(placeholder='Introduzca un número', description='ply_No_cards:')
btn = widgets.Button(description='Predecir', button_style='success')
out = widgets.Output()

def on_click(_):
    with out:
        clear_output()
        # Crear un Data frame con las features que he entrenado el modelo
        X_manual = pd.DataFrame([{
            "sumofcards": w_sumofcards.value,
            "dealcard1": w_dealcard1.value,
            "ply2cardsum": w_ply2cardsum.value,
            "ply_No_cards": w_ply_No_cards.value
        }])

        # Predicción con las features aportadas al modelo
        pred = model.predict(X_manual)[0]
        print(f"Predicción: {label_map[int(pred)]}  (0=Loss, 1=Push, 2=Win)")

        # Extraer la probabilidad de que suceda loss, push o win con las features proporcionadas
        proba, classes, aproximadas = get_probabilities_safe(model, X_manual)
        classes = np.asarray(classes).astype(int)
        labels = [label_map.get(c, str(c)) for c in classes]

        # Diccionario {Label: %}
        perc = (proba[0] * 100).round(2)
        perc_dict = dict(zip(labels, perc))

        # DataFrame Loss/Push/Win y sus probabilidades
        ordered_cols = ["Loss", "Push", "Win"]
        df_proba = pd.DataFrame([{col: perc_dict.get(col, 0.0) for col in ordered_cols}])

        titulo = "\nProbabilidades por clase (%)"
        if aproximadas:
            titulo += " (aprox.)"  
        print(titulo)
        display(df_proba.style.format("{:.2f}%"))

        loss_pct = perc_dict.get("Loss", 0.0) #Recomendacion de pedir carta o plantarse. En caso que el loss sea mayor al 50%, recomienda pedir carta.
        if loss_pct > 50:
            print("\n♣️ Recomendación: Pedir carta")
        else:  # 
            print("\n♣️ Recomendación: Plantarse")

btn.on_click(on_click)

display(widgets.VBox([w_sumofcards, w_dealcard1, w_ply2cardsum, w_ply_No_cards, btn, out]))


In [39]:
#Exportar grid_cv y best model a .pkl

joblib.dump(grid_cv, "../app_streamlit/grid_cv.pkl")
joblib.dump(model,"../app_streamlit/model.pkl")

['../app_streamlit/model.pkl']